In [3]:
using OMEinsum, Test, LinearAlgebra, Yao

In [4]:
# example 1.1: trace-permutation
code_traceperm = ein"lm,mn,nl->"

@testset "trace perm" begin
    A, B, C = rand(3, 3), rand(3, 3), rand(3, 3)
    res1 = code_traceperm(A, B, C)
    @test res1[] ≈ tr(A * B * C) 
end 

Test Summary: | Pass  Total  Time
trace perm    |    1      1  1.0s


Test.DefaultTestSet("trace perm", Any[], 1, false, false, true, 1.717228870946047e9, 1.717228871940473e9, false, "/Users/TNM/Tensor_network_methods/Qing_py/Learn_Julia/learn.ipynb")

In [5]:
# Example 1.2: SVD decomposition
code_svd = ein"ij,j,jk->ik"
@testset "SVD" begin
    A = rand(5, 3)
    U, S, V = svd(A)
    res2 = code_svd(U, S, V')
    @test res2 ≈ A
end

Test Summary: | Pass  Total  Time
SVD           |    1      1  0.4s


Test.DefaultTestSet("SVD", Any[], 1, false, false, true, 1.717228874293613e9, 1.717228874701233e9, false, "/Users/TNM/Tensor_network_methods/Qing_py/Learn_Julia/learn.ipynb")

In [6]:
# Example 1.3: Contraction order
code_order = ein"il,l,kjl,kmn,jn->im"
size_dict = uniformsize(code_order, 100)
com1 = contraction_complexity(code_order, size_dict)

opt_order1 = ein"(il,l),((kjl,kmn),jn)->im"
com2 = contraction_complexity(opt_order1, size_dict)

opt_order2 = ein"(il,l),(kjl,(kmn,jn))->im"
com3 = contraction_complexity(opt_order2, size_dict)

opt_order3 = optimize_code(code_order, size_dict, TreeSA())
com4 = contraction_complexity(opt_order3, size_dict)

println(com1)
println(com2)
println(com3)
println(com4)

Time complexity: 2^39.86313713864835
Space complexity: 2^13.287712379549449
Read-write complexity: 2^20.953119363705447
Time complexity: 2^33.23378050414179
Space complexity: 2^26.575424759098897
Read-write complexity: 2^27.59028062325597
Time complexity: 2^27.582692034590373
Space complexity: 2^19.931568569324174
Read-write complexity: 2^21.95663281043284
Time complexity: 2^27.582692034590373
Space complexity: 2^19.931568569324174
Read-write complexity: 2^21.95663281043284


In [4]:
# Example 2.1: ghz state
mutable struct IndexStore
    n::Int
    IndexStore(n::Int=0) = new(n)
end
newindex!(store::IndexStore) = store.n += 1

struct MPS{T}
    tensors::Vector{Array{T, 3}}
end

function ghz_mps(n::Int)
    t1 = zeros(ComplexF64, 1, 2, 2); t1[1, 1, 1] = t1[1, 2, 2] = 1
    t2 = zeros(ComplexF64, 2, 2, 2); t2[1, 1, 1] = t2[2, 2, 2] = 1
    t3 = zeros(ComplexF64, 2, 2, 1); t3[1, 1, 1] = t3[2, 2, 1] = 1/√2
    tensors = [t1, fill(t2, n-2)..., t3]
    return MPS(tensors)
end

function code_mps(n::Int)
    store = IndexStore()
    ixs = Vector{Int}[]
    iy = Vector{Int}()
    right = newindex!(store)
    for _ = 1:n
        left = right
        physical = newindex!(store)
        right = newindex!(store)
        push!(ixs, [left, physical, right])
        push!(iy, physical)
    end
    return EinCode(ixs, iy)
end

code_mps(5)

@testset "ghz state" begin
    n = 10
    rawmps = code_mps(n)
    ghz = ghz_mps(n)
    code = optimize_code(rawmps, uniformsize(rawmps, 2), TreeSA())
    v1 = vec(code(ghz.tensors...))
    v2 = statevec(ghz_state(n))
    @test v1 ≈ v2
end

Test Summary: | Pass  Total  Time
ghz state     |    1      1  4.0s


Test.DefaultTestSet("ghz state", Any[], 1, false, false, true, 1.717479617878921e9, 1.717479621840371e9, false, "/Users/TNM/Tensor_network_methods/Qing_py/Learn_Julia/learn.ipynb")

In [6]:
# Example 2.2: Canonical form

struct MPS{T}
    tensors::Vector{Array{T, 3}}
end

function truncated_svd(tmat::AbstractMatrix, dmax::Int, atol::Real)
    u, s, v = LinearAlgebra.svd(tmat)
    dmax = min(searchsortedfirst(s, atol, rev=true), dmax, length(s))
    return u[:, 1:dmax], s[1:dmax], v'[1:dmax, :], sum(s[dmax+1:end])
end

entropy(p::Vector) = -sum(x->x * log(x), p)

function left_canonicalize!(mps::MPS)
    for i = 1:length(mps.tensors)-1
        left, right = mps.tensors[i], mps.tensors[i+1]
        mleft, mright = reshape(left, :, size(left, 3)), reshape(right, size(right, 1), :)
        u, s, v, trunc = truncated_svd(mleft * mright, size(mleft, 2), 0)
        @info "entropy = $(entropy(s .^ 2))"
        mps.tensors[i] = reshape(u, size(left, 1), size(left, 2), size(u, 2))
        mps.tensors[i+1] = reshape(Diagonal(s) * v, size(v, 1), size(right, 2), size(right, 3))
        mps.tensors[i+1] = permutedims(mps.tensors[i+1], (2, 1, 3))
    end
    return mps
end

@testset "canonical form" begin
    n = 5
    ghz = ghz_mps(n)
    left_canonicalize!(ghz)
    left_env = ones(ComplexF64, 1, 1)
    for i = 1:n
        left_env = ein"(ab,aij),bik->jk"(left_env, conj.(ghz.tensors[i]), ghz.tensors[i])
        @test left_env ≈ Matrix{ComplexF64}(I, size(left_env)...)
    end
end

Test Summary:  | Pass  Total  Time
canonical form |    5      5  0.0s


┌ Info: entropy = -0.0
└ @ Main /Users/TNM/Tensor_network_methods/Qing_py/Learn_Julia/learn.ipynb:20
┌ Info: entropy = -0.0
└ @ Main /Users/TNM/Tensor_network_methods/Qing_py/Learn_Julia/learn.ipynb:20
┌ Info: entropy = -0.0
└ @ Main /Users/TNM/Tensor_network_methods/Qing_py/Learn_Julia/learn.ipynb:20
┌ Info: entropy = 0.6931471805599454
└ @ Main /Users/TNM/Tensor_network_methods/Qing_py/Learn_Julia/learn.ipynb:20


Test.DefaultTestSet("canonical form", Any[], 5, false, false, true, 1.717479852121947e9, 1.717479852122712e9, false, "/Users/TNM/Tensor_network_methods/Qing_py/Learn_Julia/learn.ipynb")

In [1]:
using ITensors, ITensorMPS
let
  N = 10
  sites = siteinds("S=1/2",N)

  os = OpSum()
  for j=1:N-1
    os += "Sz",j,"Sz",j+1
    os += 1/2,"S+",j,"S-",j+1
    os += 1/2,"S-",j,"S+",j+1
  end
  H = MPO(os,sites)

  psi0 = random_mps(sites;linkdims=10)

  nsweeps = 5
  maxdim = [16,24,36,44,100]
  cutoff = [1E-10]

  energy,psi = dmrg(H,psi0;nsweeps,maxdim,cutoff)

  return 
end

After sweep 1 energy=-4.2575981587225815  maxlinkdim=16 maxerr=8.85E-06 time=6.929
After sweep 2 energy=-4.258035206805498  maxlinkdim=22 maxerr=1.33E-10 time=0.005
After sweep 3 energy=-4.258035206805349  maxlinkdim=20 maxerr=9.22E-11 time=0.005
After sweep 4 energy=-4.258035206805355  maxlinkdim=20 maxerr=9.16E-11 time=0.008
After sweep 5 energy=-4.258035206805353  maxlinkdim=20 maxerr=9.16E-11 time=0.005


In [2]:
using ITensors, ITensorMPS

let
  N = 10
  cutoff = 1E-8
  tau = 0.1
  ttotal = 5.0

  # Make an array of 'site' indices
  s = siteinds("S=1/2", N; conserve_qns=true)

  # Make gates (1,2),(2,3),(3,4),...
  gates = ITensor[]
  for j in 1:(N - 1)
    s1 = s[j]
    s2 = s[j + 1]
    hj =
      op("Sz", s1) * op("Sz", s2) +
      1 / 2 * op("S+", s1) * op("S-", s2) +
      1 / 2 * op("S-", s1) * op("S+", s2)
    Gj = exp(-im * tau / 2 * hj)
    push!(gates, Gj)
  end
  # Include gates in reverse order too
  # (N,N-1),(N-1,N-2),...
  append!(gates, reverse(gates))

  # Initialize psi to be a product state (alternating up and down)
  psi = MPS(s, n -> isodd(n) ? "Up" : "Dn")

  c = div(N, 2) # center site

  # Compute and print <Sz> at each time step
  # then apply the gates to go to the next time
  for t in 0.0:tau:ttotal
    Sz = expect(psi, "Sz"; sites=c)
    println("$t $Sz")

    t≈ttotal && break

    psi = apply(gates, psi; cutoff)
    normalize!(psi)
  end

  return
end

0.0 0.5
0.1 0.4950166442955186
0.2 0.48026435362997627
0.3 0.4563259916566175
0.4 0.4241383988594508
0.5 0.38494194097057144
0.6 0.3402152047579973
0.7 0.2915987689236375
0.8 0.24081311879285608
0.9 0.18957612125454035
1.0 0.1395246678818202
1.1 0.09214495065942671
1.2 0.04871482415839579
1.3 0.010261107057733107
1.4 -0.022467094666772578
1.5 -0.049008706782350676
1.6 -0.06918539863434985
1.7 -0.08308234853686178
1.8 -0.09101776269283178
1.9 -0.09350364537194998
2.0 -0.09120106796651971
2.1 -0.08487275547396363
2.2 -0.07533658661333376
2.3 -0.06342201454534535
2.4 -0.04993156020519557
2.5 -0.03560904884218848
2.6 -0.02111550797567908
2.7 -0.007013147325221143
2.8 0.006243135575112847
2.9 0.018307094777279057
3.0 0.028936597972466106
3.1 0.03798438412251904
3.2 0.04538551607210699
3.3 0.051143084742723915
3.4 0.05531336320565637
3.5 0.057991485680458935
3.6 0.05929866653052091
3.7 0.059371318230503234
3.8 0.058352525697764376
3.9 0.05638579564168852
4.0 0.05361094607230759
4.1 0.0501617

In [38]:
let
    N = 10
    cutoff = 1E-8
    tau = 0.1
    ttotal = 5.0
  
    # Make an array of 'site' indices
    s = siteinds("S=1/2", N; conserve_qns=true)
  
    # Make gates (1,2),(2,3),(3,4),...
    gates = ITensor[]
    for j in 1:(N - 1)
      s1 = s[j]
      s2 = s[j + 1]
      hj =
        op("Sz", s1) * op("Sz", s2) +
        1 / 2 * op("S+", s1) * op("S-", s2) +
        1 / 2 * op("S-", s1) * op("S+", s2)
      Gj = exp(-im * tau / 2 * hj)
      push!(gates, Gj)
    end
    # Include gates in reverse order too
    # (N,N-1),(N-1,N-2),...
    append!(gates, reverse(gates))
  
    # Initialize psi to be a product state (alternating up and down)
    psi = MPS(s, n -> isodd(n) ? "Up" : "Dn")
  
    c = div(N, 2) # center site
  
    # Compute and print <Sz> at each time step
    # then apply the gates to go to the next time
    for t in 0.0:tau:ttotal
      Sz = expect(psi, "Sz"; sites=c)
      println("$t $Sz")
  
      t≈ttotal && break
  
      psi = apply(gates, psi; cutoff)
      
      normalize!(psi)
      bits = sample!(psi, 1:10)
      @show bits
    end
  
    return
  end

0.0 0.5


MethodError: MethodError: no method matching sample!(::MPS, ::UnitRange{Int64})

Closest candidates are:
  sample!(::MPS)
   @ ITensors ~/.julia/packages/ITensors/iifFp/src/lib/ITensorMPS/src/mps.jl:625


In [19]:

using LinearAlgebra

In [30]:


# 2. 定义初态
function define_initial_state(nqubits)
    # Define a 4-qubit system, initialize to |0000>
    state = [1, 0] # This is |0> state for each qubit
    psi0 = state
    for i in 1:nqubits-1
        psi0 = kron(psi0, state)
    end
    return psi0
end

# 3. 将初态表示为MPS
function initial_state_mps(nqubits)
    # Convert the state to an MPS
    sites = siteinds("Qubit", nqubits)
    psi_mps = productMPS(sites, "0")
    return psi_mps, sites
end

# 4. 定义哈密顿量
function define_hamiltonian(sites)
    # Example Hamiltonian: simple nearest-neighbor interaction
    ampo = AutoMPO()
    for j in 1:3
        add!(ampo, "Sz", j, "Sz", j+1)
        add!(ampo, "Sx", j)
        add!(ampo, "Sx", j+1)
    end
    H = MPO(ampo, sites)
    return H
end

# 5. 演化初态
function evolve_state(psi, H, tau)
    # Perform time evolution using the time evolution operator
    step_size = 0.1
    num_steps = Int(tau / step_size)
    psi_evolved = psi
    
    return psi_evolved
end

# 6. 测量第一个qubit
function measure_first_qubit(psi, sites)
    # Measure the first qubit in the Z basis
    prob_up = inner(psi, "Sz", sites[1], psi)
    rand_num = rand()
    return rand_num < prob_up ? 1 : 0
end

# 主程序
function main()
    

    nqubits = 4
    tau = 1.0

    psi, sites = initial_state_mps(nqubits)
    H = define_hamiltonian(sites)

    psi_evolved = evolve_state(psi, H, tau)
    result = measure_first_qubit(psi_evolved, sites)

    println("Measurement result: ", result)
end

main()


MethodError: MethodError: no method matching inner(::MPS, ::String, ::Index{Int64}, ::MPS)

Closest candidates are:
  inner(!Matched::MPO, !Matched::MPS, !Matched::MPO, ::MPS)
   @ ITensors ~/.julia/packages/ITensors/iifFp/src/lib/ITensorMPS/src/mpo.jl:524
  inner(::MPS, !Matched::MPO, !Matched::MPS; kwargs...)
   @ ITensors ~/.julia/packages/ITensors/iifFp/src/lib/ITensorMPS/src/mpo.jl:449
  inner(::MPS, !Matched::ITensors.LazyApply.Applied{typeof(product), Tuple{MPO, MPS}})
   @ ITensors ~/.julia/packages/ITensors/iifFp/src/lib/ITensorMPS/src/mpo.jl:451
  ...


In [1]:
# Example 2.2: Canonical form
function truncated_svd(tmat::AbstractMatrix, dmax::Int, atol::Real)
    u, s, v = LinearAlgebra.svd(tmat)
    dmax = min(searchsortedfirst(s, atol, rev=true), dmax, length(s))
    return u[:, 1:dmax], s[1:dmax], v'[1:dmax, :], sum(s[dmax+1:end])
end

entropy(p::Vector) = -sum(x->x * log(x), p)

function left_canonicalize!(mps::MPS)
    for i = 1:length(mps.tensors)-1
        left, right = mps.tensors[i], mps.tensors[i+1]
        mleft, mright = reshape(left, :, size(left, 3)), reshape(right, size(right, 1), :)
        u, s, v, trunc = truncated_svd(mleft * mright, size(mleft, 2), 0)
        @info "entropy = $(entropy(s .^ 2))"
        mps.tensors[i] = reshape(u, size(left, 1), size(left, 2), size(u, 2))
        mps.tensors[i+1] = reshape(Diagonal(s) * v, size(v, 1), size(right, 2), size(right, 3))
        mps.tensors[i+1] = permutedims(mps.tensors[i+1], (2, 1, 3))
    end
    return mps
end

@testset "canonical form" begin
    n = 5
    ghz = ghz_mps(n)
    left_canonicalize!(ghz)
    left_env = ones(ComplexF64, 1, 1)
    for i = 1:n
        left_env = ein"(ab,aij),bik->jk"(left_env, conj.(ghz.tensors[i]), ghz.tensors[i])
        @test left_env ≈ Matrix{ComplexF64}(I, size(left_env)...)
    end
end


UndefVarError: UndefVarError: `MPS` not defined